# Import bibliotek

In [1]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from operator import itemgetter
import time
from xgboost import XGBRegressor
import typing
SEED = 17

# Ładowanie danych

In [2]:
data=pd.read_csv("../data/data_eda.csv")
data=data.drop(columns=['Unnamed: 0'])
features=data.columns.tolist()
features.remove('stars')
target='stars'
data

,pages,stars,reviews,series,mix,character,plot,funny,lighthearted,emotional,...,author_stars,Fiction,Nonfiction,Literary,Fantasy,Crime,Social,Children,Romans,Realism
0,273,4.00,2017,0,0.44,0.51,0.02,0.27,0.37,0.91,...,4.305000,1,1,0,0,0,1,0,1,1
1,302,3.78,7330,0,0.39,0.42,0.17,0.03,0.01,0.18,...,3.670000,1,0,0,0,1,0,0,0,0
2,400,4.15,16761,0,0.51,0.39,0.08,0.02,0.01,0.88,...,0.000000,1,0,1,0,0,0,0,0,0
3,459,4.16,2128,1,0.48,0.10,0.40,0.04,0.02,0.07,...,0.000000,1,0,0,1,0,0,0,0,0
4,160,3.65,6634,1,0.28,0.16,0.54,0.92,0.73,0.00,...,4.115000,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6925,432,4.15,30643,0,0.48,0.05,0.46,0.00,0.00,0.40,...,3.856667,1,0,0,1,1,0,0,0,0
6926,352,3.62,1058,0,0.55,0.13,0.30,0.15,0.10,0.25,...,3.700000,1,0,0,1,0,1,0,0,0
6927,535,3.88,30975,1,0.45,0.08,0.45,0.14,0.19,0.31,...,3.870000,1,0,0,1,0,0,1,0,0
6928,472,3.88,5914,1,0.64,0.12,0.22,0.07,0.00,0.36,...,3.660000,1,0,0,1,0,0,1,0,0


$\text{Podział danych na zbiór treningowy i testowy}$

In [3]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=SEED)

In [4]:
def perform_cv(X: pd.DataFrame, y: pd.Series, algorithm: typing.Any, cv: typing.Any = KFold(n_splits=5, shuffle=True, random_state=SEED), metric: typing.Any = mean_squared_error) -> typing.List[float]:
    """
    Perform cross-validation and return list of scores
    
    Args:
        X (pd.DataFrame): input data
        y (pd.Series): target data
        algorithm (typing.Any): algorithm to use for training and prediction
        cv (typing.Any): cross-validation strategy
        metric (typing.Any): metric to use for evaluation
    
    Returns:
        typing.List[float]: list of scores in order: train_scores, validation_scores
    """
    train_scores = []
    validation_scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        algorithm.fit(X_train, y_train)
        y_train_pred = algorithm.predict(X_train)
        y_val_pred = algorithm.predict(X_val)
        train_scores.append(metric(y_train, y_train_pred, squared=False))
        validation_scores.append(metric(y_val, y_val_pred, squared=False))
    return np.mean(train_scores), np.mean(validation_scores)

def evaluation(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, algorithm: typing.Any, metric: typing.Any = mean_squared_error) -> typing.Tuple[float, float, np.ndarray]:
    """
    Train the algorithm on the train data and evaluate on the train and test data
    
    Args:
        X_train (pd.DataFrame): input train data
        y_train (pd.Series): target train data
        X_test (pd.DataFrame): input test data
        y_test (pd.Series): target test data
        algorithm (typing.Any): algorithm to use for training and prediction
        metric (typing.Any): metric to use for evaluation
    
    Returns:
        typing.Tuple[float, float, np.ndarray]: train_score, test_score, predictions on test data
    """
    algorithm.fit(X_train, y_train)
    y_train_pred = algorithm.predict(X_train)
    y_test_pred = algorithm.predict(X_test)
    train_results = metric(y_train, y_train_pred, squared=False)
    test_results = metric(y_test, y_test_pred, squared=False)
    return train_results, test_results, y_test_pred

## Model bez feature engineeringu

In [5]:
model = XGBRegressor(random_state=SEED, n_jobs=-1)
train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.06206850002022659
Validation RMSE: 0.19808813044143286


In [6]:
from sklearn import metrics
features=[]
df=pd.DataFrame()
def CVTestXGB(nFolds = 5, randomState=2024, debug=False, features=features, *args, **kwargs):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)
    # Listy z wynikami
    testResults = []
    trainResults = []
    predictions = []
    indices = []
    # Pętla walidująca model na kolejnych foldach
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = XGBRegressor(*args, **kwargs)
        # Trenowanie modelu
        clf.fit(df.iloc[train][features], df.iloc[train][target])
        # Przygotowanie prognoz dla zbioru treningowego i testowego
        predsTrain = clf.predict(df.iloc[train][features])
        preds = clf.predict(df.iloc[test][features])
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        # Policzenie RMSE dla foldów
        trainScore = metrics.mean_squared_error(df.iloc[train][target], predsTrain)**0.5
        testScore = metrics.mean_squared_error(df.iloc[test][target], preds)**0.5
        # Zapisanie wyników dla foldów
        trainResults.append(trainScore)
        testResults.append(testScore)
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train RMSE:", trainScore,
                  "Valid RMSE:", testScore)
        
    return trainResults, testResults, predictions, indices
df=pd.read_csv("../data/data_eda.csv")
df=df.drop(columns=['Unnamed: 0'])
features=df.columns.tolist()
features.remove('stars')
target='stars'
trainResults, testResults, predictions, indices = CVTestXGB(nFolds=5, randomState=SEED, debug=True, features=features)
print(np.mean(testResults))

Train RMSE: 0.0716487447885672 Valid RMSE: 0.19973691488499062
Train RMSE: 0.07188839435902632 Valid RMSE: 0.19262187347958723
Train RMSE: 0.07570376617210013 Valid RMSE: 0.19117966471671766
Train RMSE: 0.07493703710297953 Valid RMSE: 0.20097964298490753
Train RMSE: 0.0729050190576533 Valid RMSE: 0.19670199834049415
0.19624401888133944


In [7]:
print(np.mean(testResults))

0.19624401888133944


## Model z interakcjami

In [8]:
data_interactions=pd.read_csv("../data/data_add.csv")
data_interactions=data_interactions.drop(columns=['Unnamed: 0'])
features_interactions=data_interactions.columns.tolist()
features_interactions.remove('stars')
train_data_interactions, test_data_interactions = train_test_split(data_interactions, test_size=0.2, random_state=SEED)
model = XGBRegressor(random_state=SEED, n_jobs=-1)
train_scores, validation_scores = perform_cv(train_data_interactions[features_interactions], train_data_interactions[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.05993165594028156
Validation RMSE: 0.19920281798639167


## Model z transformacją zmiennych

In [9]:
data_transformations=pd.read_csv("../data/data_fe.csv")
data_transformations=data_transformations.drop(columns=['Unnamed: 0'])
features_transformations=data_transformations.columns.tolist()
features_transformations.remove('stars')
train_data_transformations, test_data_transformations = train_test_split(data_transformations, test_size=0.2, random_state=SEED)
model = XGBRegressor(random_state=SEED, n_jobs=-1)
train_scores, validation_scores = perform_cv(train_data_transformations[features_transformations], train_data_transformations[target], model)
print("Train RMSE:", train_scores)
print("Validation RMSE:", validation_scores)

Train RMSE: 0.06740134432400799
Validation RMSE: 0.20647785102519228


Widzimy, że dla takiego silnego modelu najlepszy jest zbiór danych bez interakcji i transformacji zmiennych.

Moc predykcji jest już lepsza od poprzedni modeli, ale postramy się ją poprawić poprzez tuning parametrów

# Tuning hiperparametrów XGBoost

1. Głębokość drzewa

In [10]:
for max_depth in range(3, 10):
    model = XGBRegressor(max_depth=max_depth, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Max depth: {}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(max_depth, train_scores, validation_scores))

Max depth: 3; RMSE train: 0.15472; RMSE validation: 0.19491
Max depth: 4; RMSE train: 0.12600; RMSE validation: 0.19312
Max depth: 5; RMSE train: 0.09536; RMSE validation: 0.19459
Max depth: 6; RMSE train: 0.06207; RMSE validation: 0.19809
Max depth: 7; RMSE train: 0.03479; RMSE validation: 0.20041
Max depth: 8; RMSE train: 0.01770; RMSE validation: 0.20327
Max depth: 9; RMSE train: 0.00741; RMSE validation: 0.20310


Najlepszy wynik dla głębokości drzewa równej 4.

2. Subsample - udział wierszy w każdym drzewie

In [11]:
for subsample in np.linspace(0.1, 1, 10):
    model = XGBRegressor(max_depth=4, subsample=subsample, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Subsample: {:.1f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(subsample, train_scores, validation_scores))

Subsample: 0.1; RMSE train: 0.19885; RMSE validation: 0.24957
Subsample: 0.2; RMSE train: 0.15881; RMSE validation: 0.21682
Subsample: 0.3; RMSE train: 0.14388; RMSE validation: 0.20719
Subsample: 0.4; RMSE train: 0.13536; RMSE validation: 0.20277
Subsample: 0.5; RMSE train: 0.13084; RMSE validation: 0.19711
Subsample: 0.6; RMSE train: 0.12781; RMSE validation: 0.19553
Subsample: 0.7; RMSE train: 0.12618; RMSE validation: 0.19329
Subsample: 0.8; RMSE train: 0.12556; RMSE validation: 0.19357
Subsample: 0.9; RMSE train: 0.12407; RMSE validation: 0.19385
Subsample: 1.0; RMSE train: 0.12600; RMSE validation: 0.19312


Najlepszy wynik w sytuacji, gdy wszystkie wiersze są brane pod uwagę.

3. colsample_bytree - udział kolumn w każdym drzewie

In [12]:
for colsample_bytree in np.linspace(0.1, 1, 10):
    model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=colsample_bytree, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Colsample by tree: {:.1f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(colsample_bytree, train_scores, validation_scores))

Colsample by tree: 0.1; RMSE train: 0.15362; RMSE validation: 0.20193
Colsample by tree: 0.2; RMSE train: 0.13970; RMSE validation: 0.19617
Colsample by tree: 0.3; RMSE train: 0.13433; RMSE validation: 0.19445
Colsample by tree: 0.4; RMSE train: 0.13239; RMSE validation: 0.19290
Colsample by tree: 0.5; RMSE train: 0.12923; RMSE validation: 0.19258
Colsample by tree: 0.6; RMSE train: 0.12884; RMSE validation: 0.19227
Colsample by tree: 0.7; RMSE train: 0.12931; RMSE validation: 0.19349
Colsample by tree: 0.8; RMSE train: 0.12784; RMSE validation: 0.19324
Colsample by tree: 0.9; RMSE train: 0.12784; RMSE validation: 0.19362
Colsample by tree: 1.0; RMSE train: 0.12600; RMSE validation: 0.19312


Rezultaty sugerują, że procent kolumn w każdym drzewie powinien wynosić 50%.

4. colsample_bylevel - udział kolumn w każdym poziomie drzewa

In [13]:
for colsample_bylevel in np.linspace(0.1, 1, 10):
    model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=colsample_bylevel, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Colsample by level: {:.1f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(colsample_bylevel, train_scores, validation_scores))

Colsample by level: 0.1; RMSE train: 0.16347; RMSE validation: 0.20246
Colsample by level: 0.2; RMSE train: 0.14417; RMSE validation: 0.19405
Colsample by level: 0.3; RMSE train: 0.14062; RMSE validation: 0.19257
Colsample by level: 0.4; RMSE train: 0.13757; RMSE validation: 0.19230
Colsample by level: 0.5; RMSE train: 0.13594; RMSE validation: 0.19357
Colsample by level: 0.6; RMSE train: 0.13452; RMSE validation: 0.19401
Colsample by level: 0.7; RMSE train: 0.13347; RMSE validation: 0.19309
Colsample by level: 0.8; RMSE train: 0.13220; RMSE validation: 0.19251
Colsample by level: 0.9; RMSE train: 0.13093; RMSE validation: 0.19405
Colsample by level: 1.0; RMSE train: 0.12923; RMSE validation: 0.19258


Najlepszy wynik dla 40% kolumn w każdym poziomie drzewa.

Widzimy, że jednak nadal wynik na zbiorze treningowym znacząco lepszy niż testowym, więc dostosujemy parametry regularyzacji.

# Regularyzacja

Tutaj poszukamy parametru lambda (regularyzacja L2).

In [14]:
for lambda_ in [0.01, 0.1, 0.5, 1, 2, 4, 6, 8, 10, 15, 30, 50, 100]:
    model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=0.4, reg_lambda=lambda_, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Lambda: {:.2f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(lambda_, train_scores, validation_scores))

Lambda: 0.01; RMSE train: 0.13391; RMSE validation: 0.19529
Lambda: 0.10; RMSE train: 0.13402; RMSE validation: 0.19592
Lambda: 0.50; RMSE train: 0.13565; RMSE validation: 0.19428
Lambda: 1.00; RMSE train: 0.13757; RMSE validation: 0.19230
Lambda: 2.00; RMSE train: 0.13879; RMSE validation: 0.19299
Lambda: 4.00; RMSE train: 0.14111; RMSE validation: 0.19171
Lambda: 6.00; RMSE train: 0.14290; RMSE validation: 0.19272
Lambda: 8.00; RMSE train: 0.14382; RMSE validation: 0.19092
Lambda: 10.00; RMSE train: 0.14474; RMSE validation: 0.19269
Lambda: 15.00; RMSE train: 0.14815; RMSE validation: 0.19250
Lambda: 30.00; RMSE train: 0.15239; RMSE validation: 0.19227
Lambda: 50.00; RMSE train: 0.15684; RMSE validation: 0.19330
Lambda: 100.00; RMSE train: 0.16278; RMSE validation: 0.19533


Wraz ze wzrostem wartości lambda, wynik na zbiorze treningowym ulegają pogorszeniu.

Najlepszy wynik dla zbioru walidacyjnego otrzymano dla wartości lambda równej 8.

Możemy dokładniej sprawdzić wyniki dla wartości lambda z przedziału [6, 10].

In [15]:
for lambda_ in [6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10]:
    model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=0.4, reg_lambda=lambda_, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Lambda: {:.2f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(lambda_, train_scores, validation_scores))

Lambda: 6.00; RMSE train: 0.14290; RMSE validation: 0.19272
Lambda: 6.50; RMSE train: 0.14207; RMSE validation: 0.19193
Lambda: 7.00; RMSE train: 0.14284; RMSE validation: 0.19237
Lambda: 7.50; RMSE train: 0.14335; RMSE validation: 0.19262
Lambda: 8.00; RMSE train: 0.14382; RMSE validation: 0.19092
Lambda: 8.50; RMSE train: 0.14426; RMSE validation: 0.19148
Lambda: 9.00; RMSE train: 0.14452; RMSE validation: 0.19193
Lambda: 9.50; RMSE train: 0.14409; RMSE validation: 0.19287
Lambda: 10.00; RMSE train: 0.14474; RMSE validation: 0.19269


In [16]:
for lambda_ in np.linspace(7.5, 8.5, 11):
    model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=0.4, reg_lambda=lambda_, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Lambda: {:.2f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(lambda_, train_scores, validation_scores))

Lambda: 7.50; RMSE train: 0.14335; RMSE validation: 0.19262
Lambda: 7.60; RMSE train: 0.14363; RMSE validation: 0.19154
Lambda: 7.70; RMSE train: 0.14355; RMSE validation: 0.19284
Lambda: 7.80; RMSE train: 0.14346; RMSE validation: 0.19218
Lambda: 7.90; RMSE train: 0.14365; RMSE validation: 0.19093
Lambda: 8.00; RMSE train: 0.14382; RMSE validation: 0.19092
Lambda: 8.10; RMSE train: 0.14376; RMSE validation: 0.19177
Lambda: 8.20; RMSE train: 0.14413; RMSE validation: 0.19084
Lambda: 8.30; RMSE train: 0.14431; RMSE validation: 0.19219
Lambda: 8.40; RMSE train: 0.14381; RMSE validation: 0.19134
Lambda: 8.50; RMSE train: 0.14426; RMSE validation: 0.19148


Najlepsze wyniki dla lambda równego 8.

Gamma to kolejny parametr regularyzacji, który określa minimalną redukcję strat wymaganą do utworzenia nowego liścia.

In [17]:
for gamma in [0, 0.01, 0.1, 0.5, 1, 2, 5]:
    model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=0.4, reg_lambda=8, gamma=gamma, random_state=SEED, n_jobs=-1)
    train_scores, validation_scores = perform_cv(train_data[features], train_data[target], model)
    print("Gamma: {:.2f}; RMSE train: {:.5f}; RMSE validation: {:.5f}".format(gamma, train_scores, validation_scores))

Gamma: 0.00; RMSE train: 0.14382; RMSE validation: 0.19092
Gamma: 0.01; RMSE train: 0.14362; RMSE validation: 0.19228
Gamma: 0.10; RMSE train: 0.15066; RMSE validation: 0.19182
Gamma: 0.50; RMSE train: 0.18415; RMSE validation: 0.20153
Gamma: 1.00; RMSE train: 0.20019; RMSE validation: 0.21019
Gamma: 2.00; RMSE train: 0.21285; RMSE validation: 0.21891
Gamma: 5.00; RMSE train: 0.22834; RMSE validation: 0.23259


Jak jednak widać, wzrost parametru gamma prowadzi do znacznych spadków wyników dla zarówno zbioru treningowego i walidacyjnego.

Tym samym zachowana zostanie domyślna wartość gamma - 0.

## Selekcja zmiennych

Do selekcji zmiennych wykorzystany 

In [18]:
import typing
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors, KDTree
from scipy.special import digamma

class MutualInformation:
    """Class for calculating mutual information between features and target."""

    def __init__(self, n_neighbors: int = 3, random_state: int = 17) -> None:
        """Initialize the class.

        Args:
            n_neighbors (int, default=3): number of neighbors to consider for continuous features.
            random_state (int, default=17): random seed for reproducibility.
        """
        self.n_neighbors_ = n_neighbors
        self.random_state_ = random_state
        np.random.seed(17)

    def check_X(
        self, X: typing.Union[pd.DataFrame, pd.Series, np.ndarray]
    ) -> np.ndarray:
        """Check if X is pandas DataFrame, pandas Series or numpy array and convert it to numpy array.

        Args:
            X: (Union[pd.DataFrame, pd.Series, np.ndarray]): input data.

        Returns:
            X: (np.ndarray): converted input data.
        """
        if (
            not isinstance(X, pd.DataFrame)
            and not isinstance(X, pd.Series)
            and not isinstance(X, np.ndarray)
        ):
            raise TypeError(
                "Wrong type of X. It should be pandas DataFrame, pandas Series, numpy array."
            )
        X = np.array(X)
        if X.ndim == 1:
            X = X[None, :]
        return X

    def check_y(
        self, y: typing.Union[pd.DataFrame, pd.Series, np.ndarray]
    ) -> np.ndarray:
        """Check if y is pandas DataFrame, pandas Series or numpy array and convert it to numpy array.

        Args:
            y: (Union[pd.DataFrame, pd.Series, np.ndarray]): target data.

        Returns:
            y: (np.ndarray): converted target data.
        """
        if (
            not isinstance(y, pd.DataFrame)
            and not isinstance(y, pd.Series)
            and not isinstance(y, np.ndarray)
        ):
            raise TypeError(
                "Wrong type of y. It should be pandas DataFrame, pandas Series, numpy array."
            )
        y = np.array(y)
        if y.ndim != 1:
            y = y.squeeze()
        return y

    def check_discrete_features(
        self,
        X: np.ndarray,
        discrete_features: typing.Union[bool, np.ndarray, typing.List],
    ) -> np.ndarray:
        """Check if discrete_features is bool, numpy array or list and convert it to numpy array.

        Args:
            X: (np.ndarray): input data.
            discrete_features: (Union[bool, np.ndarray, List]): discrete features.

        Returns:
            discrete_features: (np.ndarray): converted discrete features.
        """
        if (
            not isinstance(discrete_features, bool)
            and not isinstance(discrete_features, np.ndarray)
            and not isinstance(discrete_features, typing.List)
        ):
            raise TypeError(
                "Wrong type of discrete_features. It should be bool, numpy array or List."
            )
        if isinstance(discrete_features, np.ndarray) or isinstance(
            discrete_features, typing.List
        ):
            if len(discrete_features) > X.shape[1]:
                raise ValueError(
                    "Length of discrete_features list should be less or equal to the number of features in X."
                )
            if all(isinstance(i, np.bool_) for i in discrete_features):
                print("A")
                return np.array(discrete_features)
            return np.array(
                [True if i in discrete_features else False for i in range(X.shape[1])]
            )
        elif discrete_features == True:
            return np.array([True for i in range(X.shape[1])])
        return np.array([False for i in range(X.shape[1])])

    def check_discrete_target(self, target_discrete: bool) -> bool:
        """Check if target_discrete is bool.

        Args:
            target_discrete: (bool): target type.

        Returns:
            target_discrete: (bool): target type.
        """
        if not isinstance(target_discrete, bool):
            raise TypeError("Wrong type of target_discrete. It should be bool.")
        return target_discrete

    def fit(
        self,
        X: typing.Union[pd.DataFrame, pd.Series, np.ndarray],
        y: typing.Union[pd.DataFrame, pd.Series, np.ndarray],
        discrete_features: typing.Union[bool, np.ndarray, typing.List],
        target_discrete: bool = True,
    ) -> np.ndarray:
        """Calculate mutual information between features and target.

        Args:
            X: (Union[pd.DataFrame, pd.Series, np.ndarray]): input data.
            y: (Union[pd.DataFrame, pd.Series, np.ndarray]): target data.
            discrete_features: (Union[bool, np.ndarray, List]): discrete features.
            target_discrete: (bool, default=True): target type.

        Returns:
            (np.ndarray): mutual information between features and target.
        """
        X = self.check_X(X=X)
        y = self.check_y(y=y)
        self.discrete_features_ = self.check_discrete_features(
            X=X, discrete_features=discrete_features
        )
        self.discrete_target_ = self.check_discrete_target(
            target_discrete=target_discrete
        )
        return self.estimate_mi(X, y)

    def estimate_mi(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        """Estimate mutual information between features and target.

        Args:
            X: (np.ndarray): input data.
            y: (np.ndarray): target data.

        Returns:
            (np.ndarray): mutual information between features and target.
        """
        X, y = self.prepare_data(X, y)
        return np.array(
            [
                self.calculate_mi(X[:, i], y, self.discrete_features_[i])
                for i in range(X.shape[1])
            ]
        )

    def prepare_data(
        self, X: np.ndarray, y: np.ndarray
    ) -> typing.Tuple[np.ndarray, np.ndarray]:
        """Prepare data for mutual information calculation.

        Args:
            X: (np.ndarray): input data.
            y: (np.ndarray): target data.

        Returns:
            (np.ndarray, np.ndarray): prepared input and target data.
        """
        discrete_mask = self.discrete_features_.copy()
        continous_mask = ~discrete_mask
        X = X.astype(np.float64)
        X[:, continous_mask] = X[:, continous_mask] / np.std(
            X[:, continous_mask], axis=0
        )
        # Add small noise to continuous features
        means = np.maximum(1, np.mean(np.abs(X[:, continous_mask]), axis=0))
        X[:, continous_mask] += (
            1e-10
            * means
            * np.random.standard_normal(size=(X.shape[0], np.sum(continous_mask)))
        )
        y = y.astype(np.float64)
        if self.discrete_target_ is False:
            y = y / np.std(y)
            # Add small noise to continuous features
            y += (
                1e-10
                * np.maximum(1, np.mean(np.abs(y)))
                * np.random.standard_normal(size=(X.shape[0],))
            )
        return X, y

    def calculate_mi(
        self, x: np.ndarray, y: np.ndarray, discrete_feature: bool
    ) -> float:
        """Choose the right mutual information calculation method.

        Args:
            x: (np.ndarray): feature data.
            y: (np.ndarray): target data.
            discrete_feature: (bool): feature type.

        Returns:
            (float): mutual information between feature and target.
        """
        if discrete_feature and self.discrete_target_:
            return self.mutual_information_dd(x, y)
        elif discrete_feature and not self.discrete_target_:
            return self.mutual_information_cd(y, x)
        elif not discrete_feature and self.discrete_target_:
            return self.mutual_information_cd(x, y)
        else:
            return self.mutual_information_cc(x, y)

    def mutual_information_dd(self, x: np.ndarray, y: np.ndarray) -> float:
        """Calculate mutual information between discrete feature and discrete target.

        Args:
            x: (np.ndarray): feature data.
            y: (np.ndarray): target data.

        Returns:
            (float): mutual information between discrete feature and target.
        """
        crosstab = np.array(pd.crosstab(x, y, margins=True))
        crosstab = crosstab / crosstab[-1, -1]
        mi = 0
        for i in range(crosstab.shape[0] - 1):
            for j in range(crosstab.shape[1] - 1):
                if crosstab[i, j] != 0:
                    mi += crosstab[i, j] * np.log(
                        crosstab[i, j] / (crosstab[i, -1] * crosstab[-1, j])
                    )
        return mi

    def mutual_information_cd(
        self, continous: np.ndarray, discrete: np.ndarray
    ) -> float:
        """Calculate mutual information between continuous feature and dicrete target.

        Args:
            continous: (np.ndarray): continuous feature data.
            discrete: (np.ndarray): target data.

        Returns:
            (float): mutual information between continuous feature and target.
        """
        continous = continous.reshape(-1, 1)
        N = continous.shape[0]
        N_x = np.empty(N)
        radius = np.empty(N)
        k_all = np.empty(N)
        nearest_neighbors = NearestNeighbors()
        for label in np.unique(discrete):
            mask = discrete == label
            count = np.sum(mask)
            if count > 1:
                k = min(self.n_neighbors_, count - 1)
                k_all[mask] = k
                nearest_neighbors.set_params(n_neighbors=k)
                nearest_neighbors.fit(continous[mask].reshape(-1, 1))
                r = nearest_neighbors.kneighbors()[0]
                radius[mask] = np.nextafter(r[:, -1], 0)
            N_x[mask] = count
        mask = N_x > 1
        N_x = N_x[mask]
        k_all = k_all[mask]
        continous = continous[mask].reshape(-1, 1)
        radius = radius[mask]
        kd = KDTree(continous, metric="chebyshev")
        m_all = np.array(
            kd.query_radius(continous, radius, count_only=True, return_distance=False)
        )
        mi = (
            digamma(N)
            - np.mean(digamma(N_x))
            + np.mean(digamma(k_all))
            - np.mean(digamma(m_all))
        )
        return max(0, mi)

    def mutual_information_cc(self, x: np.ndarray, y: np.ndarray) -> float:
        """Calculate mutual information between continuous feature and continuous target.

        Args:
            x: (np.ndarray): feature data.
            y: (np.ndarray): target data.

        Returns:
            (float): mutual information between continuous feature and target.
        """
        N = x.shape[0]
        x = x.reshape(-1, 1)
        y = y.reshape(-1, 1)
        xy = np.hstack((x, y))
        nn = NearestNeighbors(metric="chebyshev", n_neighbors=self.n_neighbors_)
        nn.fit(xy)
        radius = nn.kneighbors()[0]
        radius = np.nextafter(radius[:, -1], 0)
        # fig = go.Figure()
        # fig.add_trace(go.Scatter(x=x.squeeze(), y=y.squeeze(), mode='markers', marker=dict(color='blue'), marker_size=10))
        # for i in range(x.shape[0]):
        #     fig.add_shape(type="circle", xref="x", yref="y", x0=x[i][0]-radius[i]-0.1, y0=y[i][0]-radius[i]-0.1, x1=x[i][0]+radius[i]+0.1, y1=y[i][0]+radius[i]+0.1, opacity=0.2, line=dict(color="black", width=2), fillcolor="white")
        # fig.update_layout(template="simple_white", width=600, height=600, title_text="<b>Nearest neighbors<b>", title_x=0.5, yaxis_title="y", xaxis_title="x", font=dict(family="Times New Roman",size=16,color="Black"))
        # fig.show("png")
        # fig = go.Figure()
        # fig.add_trace(go.Scatter(x=x.squeeze(), y=[0 for i in range(0, x.shape[0])], mode="markers", marker=dict(color=px.colors.qualitative.Plotly), marker_size=15, showlegend=False))
        # fig.add_trace(go.Scatter(x=[x[i][0]-radius[i] for i in range(0, x.shape[0])], y=[0 for i in range(0, len(x))], mode="markers", marker_symbol="arrow-up", marker=dict(color=px.colors.qualitative.Plotly), marker_size=15, showlegend=False))
        # fig.add_trace(go.Scatter(x=[x[i][0]+radius[i] for i in range(0, x.shape[0])], y=[0 for i in range(0, len(x))], mode="markers", marker_symbol="arrow-up", marker=dict(color=px.colors.qualitative.Plotly), marker_size=15, showlegend=False))
        # fig.update_xaxes(showgrid=False)
        # fig.update_yaxes(showgrid=False, zeroline=True, zerolinecolor='black', zerolinewidth=1, showticklabels=False)
        # fig.update_layout(plot_bgcolor='white', height=400, title_text="<b>X distances<b>", title_x=0.5, xaxis_title="x", font=dict(family="Times New Roman",size=16,color="Black"))
        # fig.show("png")
        kd = KDTree(x, metric="chebyshev")
        N_x = kd.query_radius(x, radius, count_only=True, return_distance=False)
        kd = KDTree(y, metric="chebyshev")
        N_y = kd.query_radius(y, radius, count_only=True, return_distance=False)
        mi = (
            digamma(N)
            - np.mean(digamma(N_x))
            + digamma(self.n_neighbors_)
            - np.mean(digamma(N_y))
        )
        return max(0, mi)

from sklearn.model_selection import KFold
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    mean_squared_error,
    mean_absolute_error,
)
import numpy as np
import pandas as pd
import typing


class RFECVMutualInformation(MutualInformation):
    """This class is a feature selection technique that uses Recursive Feature Elimination with Cross-Validation and Mutual Information."""

    def __init__(
        self,
        algorithm: typing.Any,
        metric: str,
        cv: KFold,
        discrete_features: typing.List[bool],
        target_discrete: bool,
    ):
        """This method initializes the class.

        Args:
            algorithm (typing.Any): algorithm that will be used for feature selection.
            metric (str): metric that will be used for evaluation.
            cv (KFold): cross-validation technique that will be used.
            discrete_features (typing.List[bool]): list of booleans that indicates whether the feature is discrete or not.
            target_discrete (bool): boolean that indicates whether the target is discrete or not.
        """
        self.algorithm = algorithm
        metrics = {
            "accuracy": [lambda y, y_pred: accuracy_score(y, y_pred), "preds"],
            "roc_auc": [lambda y, y_pred: roc_auc_score(y, y_pred), "probs"],
            "neg_mse": [lambda y, y_pred: -mean_squared_error(y, y_pred), "preds"],
            "neg_rmse": [
                lambda y, y_pred: -mean_squared_error(y, y_pred) ** 0.5,
                "preds",
            ],
            "neg_mae": [lambda y, y_pred: -mean_absolute_error(y, y_pred), "preds"],
        }
        if metric not in metrics:
            raise ValueError("Unsupported metric: {}".format(metric))
        self.metric = metric
        self.eval_metric = metrics[metric][0]
        self.metric_type = metrics[metric][1]
        self.cv = cv
        self.mutual_information = MutualInformation(n_neighbors=3, random_state=17)
        self.discrete_features_ = discrete_features
        self.discrete_target_ = target_discrete

    def check_X(
        self, X: typing.Union[pd.DataFrame, pd.Series, np.ndarray]
    ) -> np.ndarray:
        """Check if X is pandas DataFrame, pandas Series or numpy array and convert it to numpy array.

        Args:
            X: (Union[pd.DataFrame, pd.Series, np.ndarray]): input data.

        Returns:
            X: (np.ndarray): converted input data.
        """
        if (
            not isinstance(X, pd.DataFrame)
            and not isinstance(X, pd.Series)
            and not isinstance(X, np.ndarray)
        ):
            raise TypeError(
                "Wrong type of X. It should be pandas DataFrame, pandas Series, numpy array."
            )
        X = np.array(X)
        if X.ndim == 1:
            X = X[None, :]
        return X

    def check_y(
        self, y: typing.Union[pd.DataFrame, pd.Series, np.ndarray]
    ) -> np.ndarray:
        """Check if y is pandas DataFrame, pandas Series or numpy array and convert it to numpy array.

        Args:
            y: (Union[pd.DataFrame, pd.Series, np.ndarray]): target data.

        Returns:
            y: (np.ndarray): converted target data.
        """
        if (
            not isinstance(y, pd.DataFrame)
            and not isinstance(y, pd.Series)
            and not isinstance(y, np.ndarray)
        ):
            raise TypeError(
                "Wrong type of y. It should be pandas DataFrame, pandas Series, numpy array."
            )
        y = np.array(y)
        if y.ndim != 1:
            y = y.squeeze()
        return y

    def check_for_object_columns(self, X: np.ndarray) -> np.ndarray:
        """Check if X contains object columns and convert it to numeric data.

        Args:
            X: (np.ndarray): input data.

        Returns:
            X: (np.ndarray): converted input data.
        """
        X = pd.DataFrame(X)
        if X.select_dtypes(include=np.number).shape[1] != X.shape[1]:
            raise TypeError(
                "Your data contains object or string columns. Numeric data is obligated."
            )
        return np.array(X)

    def fit(
        self,
        X: typing.Union[pd.DataFrame, pd.Series, np.ndarray],
        y: typing.Union[pd.DataFrame, pd.Series, np.ndarray],
        verbose: bool = False,
    ) -> None:
        """Fit the Recursive Feature Elimination with Cross-Validation and Mutual Information to the data.

        Args:
            X (Union[pd.DataFrame, pd.Series, np.ndarray]): input data.
            y (Union[pd.DataFrame, pd.Series, np.ndarray]): target data.
            verbose (bool): boolean that indicates whether the information about the process will be printed out.
        """
        X = self.check_X(X)
        X = self.check_for_object_columns(X)
        y = self.check_y(y)
        self.mi_ = list(
            self.mutual_information.fit(
                X,
                y,
                discrete_features=self.discrete_features_,
                target_discrete=self.discrete_target_,
            )
        )
        self.indices_of_best_, self.support_ = self.perform_rfecv(X, y, verbose)

    def perform_rfecv(
        self, X: np.ndarray, y: np.ndarray, verbose: bool
    ) -> typing.Tuple[np.ndarray, np.ndarray]:
        """Perform the Recursive Feature Elimination with Cross-Validation.

        Args:
            X (np.ndarray): input data.
            y (np.ndarray): target data.
            verbose (bool): boolean that indicates whether the information about the process will be printed out.

        Returns:
            best_features (np.ndarray): indices of the best features.
            support_ (np.ndarray): boolean mask of the best features.
        """
        original_features = [i for i in range(X.shape[-1])]
        features = original_features.copy()
        best_score = -np.inf
        worst = None
        while len(features) > 1:
            if worst is not None:
                features.remove(original_features[worst])
            X_copy = X[:, features]
            score = self.perform_cv(X_copy, y)
            if score > best_score:
                if verbose == True:
                    print(
                        "After removing the worst feature: {}, score improved, because: {}>{}".format(
                            worst, score, best_score
                        )
                    )
                best_score = score
                best_features = np.setdiff1d(features, [worst])
            worst = features[np.argmin([self.mi_[i] for i in features])]
        support_ = np.array(
            [True if i in best_features else False for i in range(0, X.shape[1])]
        ).astype(bool)
        return best_features, support_

    def perform_cv(self, X: np.ndarray, y: np.ndarray) -> float:
        """This method performs cross-validation.

        Args:
            X: (np.ndarray): input data.
            y: (np.ndarray): target data.

        Returns:
            float: cross-validation score.
        """
        scores = []
        for train_index, test_index in self.cv.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            self.algorithm.fit(X_train, y_train)
            if self.metric_type == "preds":
                y_pred = self.algorithm.predict(X_test)
            else:
                y_pred = self.algorithm.predict_proba(X_test)[:, 1]
            scores.append(self.eval_metric(y_test, y_pred))
        return np.mean(scores)

In [19]:
discrete_features = ["series", "Fiction", "Nonfiction", "Literary", "Fantasy", "Crime", "Social", "Children", "Romans", "Realism"]
model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=0.4, reg_lambda=8, gamma=0, random_state=SEED, n_jobs=-1)
rfecv_mi = RFECVMutualInformation(algorithm=model, metric="neg_rmse", discrete_features=discrete_features, target_discrete=False, cv=KFold(n_splits=5, shuffle=True))
rfecv_mi.fit(train_data[features], train_data[target], verbose=True)
print("Indices of best features: {}".format(rfecv_mi.indices_of_best_))
print("Support: {}".format(rfecv_mi.support_))

After removing the worst feature: None, score improved, because: -0.1928102065020521>-inf
Indices of best features: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
Support: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


Wyniki sugerują, że model osiąga najlepsze rezultaty przy pełnym zbiorze zmiennych.

## Zapisanie modelu

In [20]:
test_indices = test_data.index
#Ewaluacja modelu
model = XGBRegressor(max_depth=4, subsample=1, colsample_bytree=0.5, colsample_bylevel=0.4, reg_lambda=8, gamma=0, random_state=SEED, n_jobs=-1)
train_results, test_results, y_test_pred = evaluation(train_data_interactions[features_interactions], train_data_interactions[target], test_data_interactions[features_interactions], test_data_interactions[target], model)
print("Train RMSE: {}".format(round(train_results, 5)))
print("Test RMSE: {}".format(round(test_results, 5)))

#Zapisanie modelu
model_XGB = {
    "name": "XGB",
    "trainResults": train_results,
    "testResults": test_results,
    "predictions": y_test_pred,
    "indices": test_indices,
}
with open("../data/model_XGB.p", "wb") as fp:
    pickle.dump(model_XGB, fp)

Train RMSE: 0.14659
Test RMSE: 0.1983


## Podsumowanie

$\text{Najlepsze wyniki walidacji krzyżowej uzyskano dla modelu bazującego na podstawowych danych}$<p>
$\text{Optymalizacja hiperparametrów oraz regularyzacja pozwoliły na poprawę wyników.}$<p>
$\text{Wyniki na zbiorze treningowym (RMSE): 0.14659}$<p>
$\text{Wyniki na zbiorze testowym (RMSE): 0.1983}$